In [6]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

llm = OpenAI(model_name="gpt-3.5-turbo-1106")
chat = ChatOpenAI()

a = llm.predict("how many planets are there")
b = chat.predict("how many planets are there")

c:\Users\wmk51\FULLSTACK-GPT\env\Lib\site-packages\langchain\llms\openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
c:\Users\wmk51\FULLSTACK-GPT\env\Lib\site-packages\langchain\llms\openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [7]:
a,b


('There are eight recognized planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.',
 'There are 8 planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.')

##3-2

In [10]:
from langchain.chat_models import ChatOpenAI
#chatmodel 생성자
chat = ChatOpenAI(temperature=0.1) ##얼마나 창의적인 답변을 요구하느ㄴ냐

In [12]:

messages = chatPromptTemplate.from_messages[
    
]

chat.predict_messages(messages)

AIMessage(content='서울과 부산 사이의 거리는 약 325km입니다. 제 이름은 원호입니다. 어떻게 도와드릴까요?')

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
# #PromptTemplate은 String을 가지고만 사용한다.
# template = PromptTemplate.from_template(
#     "What is the distance between {country_a} and {country_b}",
# )

# chat = ChatOpenAI(temperature=0.1)

# prompt = template.format(country_a = "Mexico", country_b = "Korea")
# chat.predict(prompt)

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
#PromptTemplate은 String을 가지고만 사용한다.
chat = ChatOpenAI(temperature=0.1)

#template은 큰 틀이고
template = ChatPromptTemplate.from_messages(
    [
        ("system","You are a geography expert. And you only reply in {language}."),
        ("ai","Ciao, mi chiamao {name}!"),
        ("human", "what is the distance between {country_a} and {country_b}. Also, what is your name?")
    ]
)
#prompt 조건을 만족시키게 넣어주는 역활이다
prompt = template.format_messages(
    language="Greek",
    name="Socrates",
    country_a="Mexico",
    country_b="Thailand",
)

chat.predict_messages(prompt)

AIMessage(content='Γεια σας! Το όνομά μου είναι Σωκράτης. Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα.')

In [12]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    
    def parse(self,text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    
    
p = CommaOutputParser()
p.parse("Hello, how, are, you")

['Hello', 'how', 'are', 'you']

In [16]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate

###chain 만들기 위해서 필용한 요소 chat,outputParser,template
chat = ChatOpenAI(temperature=0.1)

class CommaOutputParser(BaseOutputParser):
    
    def parse(self,text):
        items = text.strip().split(",")
        return list(map(str.strip, items))


template = ChatPromptTemplate.from_messages(
    [
        ("system","you are a generating machine. Everything you are asked will be answered with a comma separated list of max {max_items}\
         in lowercase. Do NOT reply with anything else.",),
        ("human","{question}")
    ]
)





In [17]:
## "|"만 있으면 모든지 해결된다

chain = template | chat | CommaOutputParser()
chain.invoke({
    "max_items":5,
    "question": "what are the pokemons"
})

['pikachu', 'charmander', 'bulbasaur', 'squirtle', 'jigglypuff']

In [22]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1,
                  streaming=True,
                  callbacks=[StreamingStdOutCallbackHandler()]
                  )
##첫 번쨰 chain 요리사
chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients"),
    ("human","I want to cook {cuisine} food.")
])

chef_chain = chef_prompt | chat

In [23]:
veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
      ("system","You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe.\
          If there is no alternative for a food just say you don't know how to replace it.")  ,
      ("human","{recipe}")
    ]
)

veg_chain = veg_chef_prompt | chat
## 코드 해석
## chef_chain의 결과값을 recipe라하고 veg_chain에 대입한다
final_chain = {"recipe" : chef_chain} | veg_chain
final_chain.invoke({"cuisine" :"indian" })

Great choice! Indian cuisine is known for its bold flavors and aromatic spices. Let's make a classic dish - Chicken Tikka Masala. Here's a simple recipe for you to try at home:

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tbsp lemon juice
- 2 tbsp vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1-inch piece of ginger, grated
- 1 can (14 oz) crushed tomatoes
- 2 tbsp tomato paste
- 1 tsp ground cumin
- 1 tsp ground coriander
- 1 tsp paprika
- 1 tsp garam masala
- 1/2 tsp turmeric
- Salt and pepper, to taste
- Fresh cilantro, for garnish
- Cooked rice or naan, for serving

Instructions:
1. In a bowl, combine the yogurt, lemon juice, 1 tbsp of vegetable oil, half of the minced garlic, half of the grated ginger, and a pinch of salt. Add the chicken pieces and marinate for at least 1 hour, preferably overnight.

2. Preheat the oven to 400°F (200°C). Thread the marinated chicken onto skewers and place them on

AIMessageChunk(content="To make a vegetarian version of Chicken Tikka Masala, we can replace the chicken with a suitable alternative such as paneer or tofu. Here's how you can adapt the recipe:\n\nIngredients:\n- 1 lb paneer or extra-firm tofu, cut into bite-sized pieces\n- 1 cup plain yogurt (you can use dairy-free yogurt for a vegan version)\n- 2 tbsp lemon juice\n- 2 tbsp vegetable oil\n- 1 onion, finely chopped\n- 3 cloves garlic, minced\n- 1-inch piece of ginger, grated\n- 1 can (14 oz) crushed tomatoes\n- 2 tbsp tomato paste\n- 1 tsp ground cumin\n- 1 tsp ground coriander\n- 1 tsp paprika\n- 1 tsp garam masala\n- 1/2 tsp turmeric\n- Salt and pepper, to taste\n- Fresh cilantro, for garnish\n- Cooked rice or naan, for serving\n\nInstructions:\n1. Follow the same marinating process as the original recipe, but use paneer or tofu instead of chicken. Marinate the paneer or tofu in the yogurt, lemon juice, vegetable oil, garlic, ginger, and salt for at least 1 hour.\n\n2. Instead of bak